This notebook generates a displacement adjacement matrix for stations.

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from functools import reduce
from geopy import Point
from geopy import distance

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
class AnalysisConfig:
    def __init__(self, csv_output_path='', stations_path=''):
        self.csv_output_path = csv_output_path
        self.stations_path = stations_path

In [3]:
INPUT_DIR = '../data/exports'
OUTPUT_DIR = '../data/exports/adjacency_matrix'
SHAPEFILE_DIR = '../data/shapefiles/zipcodes'
VARIABLE = 'elevation'

config_sf = AnalysisConfig(
    csv_output_path='{}/{}_sf.csv'.format(OUTPUT_DIR, VARIABLE),
    stations_path='{}/SF_ele_single station.csv'.format(INPUT_DIR),
)

config_dc = AnalysisConfig(
    csv_output_path='{}/{}_dc.csv'.format(OUTPUT_DIR, VARIABLE),
    stations_path='{}/DC_ele_single station.csv'.format(INPUT_DIR),
)

In [4]:
config = config_dc

In [5]:
def get_distance(point1, point2):
    return distance.geodesic(point1, point2).m


def get_point(station_id):
    return Point(stations_df.loc[station_id]['lat'],
                 stations_df.loc[station_id]['lng'])


def get_displacement(station_id1, station_id2):
    return get_distance(get_point(station_id1), get_point(station_id2))

def get_elevation_change(station_id1, station_id2):
    return (stations_df.loc[station_id2]['elevation'] - 
            stations_df.loc[station_id1]['elevation'])

In [6]:
stations_df = pd.read_csv(config.stations_path)
stations_df.set_index('station_id', inplace=True)
stations_df.head()

,lat,lng,DN
station_id,,,
31000,38.858970,-77.053239,14.0
31001,38.857250,-77.053402,15.0
31002,38.856408,-77.049262,10.0
31003,38.860856,-77.049425,6.0
31004,38.857866,-77.059490,16.0


In [7]:
adj_matrix = pd.DataFrame(index=stations_df.index, columns=stations_df.index)

if VARIABLE == 'displacement':
    adj_matrix = adj_matrix.apply(lambda row: row.index.to_series().apply(
        lambda col_name: get_displacement(row.name, col_name)),
                                  axis=1)
elif VARIABLE == 'elevation':
    adj_matrix = adj_matrix.apply(lambda row: row.index.to_series().apply(
        lambda col_name: get_elevation_change(row.name, col_name)),
                                  axis=1)
adj_matrix.head()

KeyError: 'elevation'

In [ ]:
adj_matrix.to_csv(config.csv_output_path)

In [ ]:
# Example for retrieving displacement from SF.
csv_adj_matrix = pd.read_csv(config.csv_output_path).set_index('station_id')
display(csv_adj_matrix.head())

csv_adj_matrix['SF-AA15']['SF-Y14']